In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 11:29:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')


In [3]:
df_green.registerTempTable('green')

/home/henriquecruzz/spark/spark-3.5.5-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
df_green_revenue = spark.sql("""
    select 
    PULocationID as zone,
    date_trunc('hour', cast(lpep_pickup_datetime as TIMESTAMP)) as hour,
    
    sum(total_amount) as amount,
    count(1) as number_of_records
    from green
    where lpep_pickup_datetime >= '2020-01-01 00:00:00'
    group by 1,2
    Order by
    zone, hour
""")

In [6]:
df_green_revenue.show(5, False)

+----+-------------------+------+-----------------+
|zone|hour               |amount|number_of_records|
+----+-------------------+------+-----------------+
|1   |2020-01-01 03:00:00|155.3 |1                |
|1   |2020-01-03 05:00:00|117.39|1                |
|1   |2020-01-26 10:00:00|85.56 |1                |
|1   |2020-01-29 18:00:00|108.36|1                |
|1   |2020-02-01 06:00:00|115.56|1                |
+----+-------------------+------+-----------------+
only showing top 5 rows



In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
    select 
    PULocationID as zone,
    date_trunc('hour', cast(tpep_pickup_datetime as TIMESTAMP)) as hour,
    
    sum(total_amount) as amount,
    count(1) as number_of_records
    from yellow
    where tpep_pickup_datetime >= '2020-01-01 00:00:00'
    group by 1,2
    Order by
    zone, hour
""")

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [11]:
## JOINS IN SPARK

In [23]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenuw = spark.read.parquet('data/report/revenue/yellow')

In [24]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_of_records', 'green_number_records')


In [25]:
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_of_records', 'yellow_number_records')


In [26]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [27]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 05:00:00|   1|        NULL|                NULL|              40.3|                    1|
|2020-01-01 07:00:00|   1|        NULL|                NULL|              0.31|                    1|
|2020-01-01 11:00:00|   1|        NULL|                NULL|              90.3|                    1|
|2020-01-01 12:00:00|   1|        NULL|                NULL|            266.66|                    3|
|2020-01-01 15:00:00|   1|        NULL|                NULL|            315.21|                    4|
|2020-01-01 18:00:00|   1|        NULL|                NULL|            400.41|                    3|
|2020-01-02 01:00:00|   1|        NULL|                NULL|             84.36|   

In [28]:
df_join \
    .repartition(20) \
    .write.parquet('data/report/revenue/total', mode='overwrite')

In [29]:
df_join = spark.read.parquet('data/report/revenue/total', mode='overwrite')

In [30]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-11-13 09:00:00|  75|226.49000000000004|                  17| 978.7399999999998|                   60|
|2021-07-24 13:00:00|  40|             12.88|                   1|             45.67|                    3|
|2020-07-31 06:00:00| 166|              9.35|                   1|            176.06|                    8|
|2020-07-08 09:00:00|  88|              NULL|                NULL|             54.72|                    2|
|2020-11-13 11:00:00| 215|             21.63|                   1|              NULL|                 NULL|
|2021-01-22 08:00:00| 246|              NULL|                NULL| 302.5200000000001|                   19|
|2020-09-06 18:00:00|  51|  

In [31]:
df_zones = spark.read.parquet('zones/')

In [33]:
df_result = df_join.join(df_zones, df_join.zone==df_zones.LocationID)

In [36]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')